# Tree Structure Provider Example

The [Tree Structure Provider](https://plugins.jetbrains.com/docs/intellij/tree-structure-view.html) allows modifying the structure of the project tree in the Project View panel.

Let's create a provider that will limit the presented files to the Plain Text files only. It will check if a Project View node represents a directory or file of the [`PlainTextFileType`](https://github.com/JetBrains/intellij-community/blob/master/platform/core-impl/src/com/intellij/openapi/fileTypes/PlainTextFileType.java) type. Otherwise, an element is not included in the results list, so only directories and plain text files are rendered.

## Enabling Kotlin Notebook Intellij Platform Integration

Firstly, let's enable IntelliJ Platform integration and load necessary libraries.

In [ ]:
%use intellij-platform

## Extension Point

We will implement the [`com.intellij.treeStructureProvider`](https://plugins.jetbrains.com/intellij-platform-explorer/extensions?extensions=com.intellij.postStartupActivity&extensions=com.intellij.treeStructureProvider) Extension Point, which allows modifying the structure of the project tree in the Project View panel.

In [ ]:
import com.intellij.ide.projectView.TreeStructureProvider
import com.intellij.ide.projectView.ViewSettings
import com.intellij.ide.projectView.impl.nodes.PsiFileNode
import com.intellij.ide.util.treeView.AbstractTreeNode
import com.intellij.openapi.fileTypes.PlainTextFileType
import org.jetbrains.annotations.Unmodifiable

class TextOnlyTreeStructureProvider : TreeStructureProvider {
    override fun modify(parent: AbstractTreeNode<*>, children: Collection<AbstractTreeNode<*>?>, settings: ViewSettings?) =
        children.filter {
            if (it is PsiFileNode) {
                val file = it.virtualFile
                file == null || file.isDirectory || file.fileType == PlainTextFileType.INSTANCE
            } else {
                true
            }
        }
}

## Registering the Extension Point

Let's create an instance of this provider.

Since the extension point `com.intellij.treeStructureProvider` is marked as *project level*, we cannot use the `registerExtension` function. To maintain the proper scope, need to use the `registerProjectExtension` that will correctly associate the extension with the current project. This function is provided by the Kotlin Notebook IntelliJ Platform Integration.

In [ ]:
import com.intellij.openapi.application.ApplicationManager
import com.intellij.openapi.extensions.ProjectExtensionPointName

runInEdt {
    val treeStructureProvider = TextOnlyTreeStructureProvider()
    registerProjectExtension("com.intellij.treeStructureProvider", treeStructureProvider)
}

## Running the Example

If we run this example, we will observe that the Project tool window and its tree will display only plain-text files and directories. For example, the `LICENSE` file will be shown.

In addition, the proper lifecycle of extensions is automatically maintained. Kotlin Notebook kernel restart will correctly dispose the project extension and restore the original full Project tool window tree.

## Summary

- Use `TreeStructureProvider` subclass to create the corresponding extension.
- Discover the proper scope of the extension point in the [IntelliJ Platform Plugin SDK](https://plugins.jetbrains.com/docs/intellij/intellij-platform-extension-point-list.html#intellijsettingssynccorexml). Project level extension points need to be registered by `registerProjectExtension` function.
- The lifecycle of the extension is automatically maintained.
